In [1]:
import cv2
import numpy as np

# ver 1

In [11]:
# Function to detect if a fingerprint area is present
def detect_fingerprint_area(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    _, thresh = cv2.threshold(blurred, 150, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        aspect_ratio = w / float(h)
        if 0.8 < aspect_ratio < 1.2 and 50 < w < 150 and 50 < h < 150:
            return "Back side"
    return "Front side"


# Path to your card image
image_path = '../../test/test_back2.jpeg'

# Detect which side of the card it is
detect_fingerprint_area(image_path)


'Back side'

# ver 2

In [9]:
import cv2

front_template = cv2.imread('../../assets/front.jpg', cv2.IMREAD_GRAYSCALE)
back_template = cv2.imread('../../assets/back.jpg', cv2.IMREAD_GRAYSCALE)    

orb = cv2.ORB_create(1000)

BFMATCHER = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)


def detect_id_card_side(image_path):
    card_image = cv2.imread(image_path,  cv2.IMREAD_GRAYSCALE) 

    des_card = orb.detectAndCompute(card_image, None)[1]

    des_front = orb.detectAndCompute(front_template, None)[1]

    des_back = orb.detectAndCompute(back_template, None)[1]

    similarity_front = get_similarity_from_desc(des_card, des_front)
    similarity_back = get_similarity_from_desc(des_card, des_back)

    return "front" if similarity_front > similarity_back else "back"
    

def get_similarity_from_desc(search, idx):
    matches = BFMATCHER.match(search, idx)
    distances = [m.distance for m in matches]
    distance = sum(distances) / len(distances)
    similarity = 1 / (1 + distance)
    return similarity

In [10]:
# Path to your card image
image_path = '../../test/test1.jpeg'

# Detect the card side
detect_id_card_side(image_path)

'front'